In [9]:
from datetime import datetime, timedelta
from pathlib import Path
import os
import sys
import numpy as np
import pandas as pd
import re
import AMBRA_Backups
from bs4 import BeautifulSoup
import AMBRA_Utils
from redcap_funcs import comp_schema_cap_db, get_project_schema

db_name = 'CAPTIVA'
db = AMBRA_Backups.database.Database(db_name)
project_name = 'CAPTIVA Data Collection'
project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name) 

def q(query, record=None):
    if any(substring in query for substring in ['UPDATE', 'INSERT', 'DELETE']):
        return db.run_insert_query(query, record)
    elif 'SELECT' in query:
        return pd.DataFrame(db.run_select_query(query, record, column_names=True))
    else:
        print('You dont have UPDATE, INSERT, DELETE or SELECT in your query my guy')

In [17]:
df = AMBRA_Backups.redcap_funcs.get_project_schema(project_name, 'form_1')
AMBRA_Backups.utils.df_to_db_table(db, df, 'CRF_Schema_RedCap')

324

In [15]:
db = AMBRA_Backups.database.Database(db_name)
project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name)

forms = [f['instrument_name'] for f in project.export_instruments()]


master_discreps = ''

for crf_name in forms:

    # redcap_variable discrepancies
    unique_data_vars = pd.DataFrame(db.run_select_query("""SELECT DISTINCT(redcap_variable) 
        FROM CRF_RedCap
        JOIN CRF_Data_RedCap
            ON CRF_RedCap.id = CRF_Data_RedCap.id_crf
        WHERE crf_name = %s""", [crf_name], column_names=True))
    print(crf_name)
    display(unique_data_vars)

    unique_data_vars = unique_data_vars['redcap_variable']
    schema_vars = pd.DataFrame(db.run_select_query("""SELECT redcap_variable FROM CRF_Schema_RedCap
                                                      WHERE crf_name = %s""", [crf_name], column_names=True))
    # display(schema_vars)

form_1


,redcap_variable
0,base_brain_status
1,base_vasc_status
2,form_1_complete


baseline_brain_crf


,redcap_variable
0,baseline_brain_crf_comments
1,baseline_brain_crf_complete
2,baseline_brain_crf_init
3,baseline_brain_crf_query
4,baseline_brain_crf_status
...,...
76,q1011(subara)
77,q1011(subdura)
78,q1012
79,q1013


baseline_vascular_crf


,redcap_variable
0,baseline_vascular_crf_complete
1,q2001
2,q2004
3,q2005
4,q2005a
...,...
159,'SYMPTOMATIC VESSEL IS RIGHT M2\r SUPERIOR DIV...
160,'SYMPTOMATIC VESSEL IS RIGHT M2 SUPERIOR DIVIS...
161,0.3; DN
162,'SYMPTOMATIC VESSEL LEFT P1 WITH DS


follow_up_brain_crf


,redcap_variable
0,follow_up_brain_crf_complete
1,q3001
2,q3001a
3,q3002
4,q3004
...,...
100,q1008_a_col1_1397c7(vert_bas_it)
101,q1008_b_col1_51ee58(r_ant_cerebral)
102,q1008_b_col2_f5b67a(int_lmca)
103,qa1_cd79ce_b02338


In [10]:
comp_schema_cap_db(db_name, project_name)

KeyError: 'redcap_variable'